In [1]:
import keras
import numpy as np
import time

from foolbox.models import KerasModel
from foolbox.attacks import AdditiveUniformNoiseAttack
from foolbox.criteria import OriginalClassProbability, Misclassification, ConfidentMisclassification

from skimage.measure import compare_ssim as ssim

Using TensorFlow backend.


In [2]:
# load data
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)
train_images = train_images / 255.0
test_images = test_images / 255.0

In [3]:
# load model
trained_model = keras.models.load_model('fashionMNIST.h5')
cnn5 = keras.models.load_model('cnn5.h5')
sub_model = KerasModel(trained_model, bounds=(0, 1), channel_axis=1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [8]:
# start attack
def aiTest(batch, shape):
    startTime = time.time()
    generate_images = []
    
    local_batch_predictions = trained_model.predict(batch)
    
    for i in range(shape[0]):
        
        if i % 50 == 0:
            print("{:3d}".format(i // 50), end=": ")
        print(i % 50, end=" ")
        if i % 50 == 49:
            print()

        
        image = batch[i]
        image_label = int(np.argmax(local_batch_predictions[i]))

        adv = None

        if adv is None:
            attack = AdditiveUniformNoiseAttack(model=sub_model, criterion=OriginalClassProbability(0.1))
            adv = attack(image, image_label)

        if adv is None:
            attack = AdditiveUniformNoiseAttack(model=sub_model, criterion=ConfidentMisclassification(0.9))
            adv = attack(image, image_label)

        if adv is None:
            attack = AdditiveUniformNoiseAttack(model=sub_model, criterion=Misclassification())
            adv = attack(image, image_label)

        if adv is None:
            adv = train_images[np.random.randint(60000)]
        
        generate_images.append(adv)
    
    endTime = time.time() - startTime
    print("\nused time: ", endTime)
    return np.array(generate_images)

In [9]:
# test pics
def get_test_batch(testNum):
    randArray = np.random.choice(10000, testNum, replace=False)
    batch = []
    for i in range(len(randArray)):
        image_id = randArray[i] 
        image = test_images[i]
        batch.append(image)
    return np.array(batch)

# self test
def self_test():
    testNum = 100
    
    batch = get_test_batch(testNum)
    remote_batch_predictions = cnn5.predict(batch)
    remote_predicted_labels = np.argmax(remote_batch_predictions, axis=1)
        

    adv_batch = aiTest(batch, (testNum, 28, 28, 1))
    remote_adv_batch_predictions = cnn5.predict(adv_batch)
    remote_adv_predicted_labels = np.argmax(remote_adv_batch_predictions, axis=1)

    attackBitmap = np.where(remote_predicted_labels != remote_adv_predicted_labels, 1, 0)
    attackSSIM = np.zeros(testNum)
    for i in range(testNum):
        original = batch[i]
        adv = adv_batch[i]
        attackSSIM[i] = ssim(original[:,:,0], adv[:,:,0])
        
    print("mean hacked rate:", np.mean(attackBitmap))
    print("mean hacked ssim:", np.dot(attackBitmap, attackSSIM)/testNum)
    print(attackBitmap * attackSSIM)

In [10]:
self_test()

  0: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 
  1: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 

used time:  152.8223819732666
mean hacked rate: 0.66
mean hacked ssim: 0.3490577335523436
[0.55611586 0.76908905 0.         0.47393404 0.14779074 0.66023526
 0.68239363 0.30213698 0.37461873 0.         0.7245273  0.59689369
 0.54757778 0.         0.50825668 0.55094376 0.         0.
 0.         0.44246062 0.32255066 0.52412845 0.         0.63208847
 0.         0.66586679 0.34487558 0.         0.66315467 0.54821967
 0.         0.29573712 0.51283178 0.5886007  0.         0.4653974
 0.         0.40528484 0.         0.58357032 0.         0.
 0.         0.66069817 0.26671618 0.         0.73215473 0.
 0.67729225 0.26625187 0.59916633 0.         0.39057976 0.
 0.65670474 0.5427688  0.         0.35444131 0.